In [1]:
import numpy as np
import numpy.ma as ma
import pymc3 as pm
import theano.tensor as tt
import os
import csv
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
import multiprocessing as mp

SMALL_SIZE = 16
MEDIUM_SIZE = 18
BIGGER_SIZE = 20

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

SEED = 350107321 # from random.org
np.random.seed(SEED)

print(plt.style.available)
plt.style.use('seaborn-white')

['Solarize_Light2', '_classic_test_patch', 'arviz-bluish', 'arviz-brownish', 'arviz-colors', 'arviz-cyanish', 'arviz-darkgrid', 'arviz-grayscale', 'arviz-greenish', 'arviz-orangish', 'arviz-plasmish', 'arviz-purplish', 'arviz-redish', 'arviz-royish', 'arviz-viridish', 'arviz-white', 'arviz-whitegrid', 'bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'seaborn', 'seaborn-bright', 'seaborn-colorblind', 'seaborn-dark', 'seaborn-dark-palette', 'seaborn-darkgrid', 'seaborn-deep', 'seaborn-muted', 'seaborn-notebook', 'seaborn-paper', 'seaborn-pastel', 'seaborn-poster', 'seaborn-talk', 'seaborn-ticks', 'seaborn-white', 'seaborn-whitegrid', 'tableau-colorblind10']


In [2]:
def cubicsolver(coef):
    
    a = coef[0]
    b = coef[1]
    c = coef[2]
    d = coef[3]

    f = findF(a, b, c)                          # Helper Temporary Variable
    g = findG(a, b, c, d)                       # Helper Temporary Variable
    h = findH(g, f)                             # Helper Temporary Variable

    i = np.sqrt(((g ** 2.0) / 4.0) - h)   # Helper Temporary Variable
    j = i ** (1 / 3.0)                      # Helper Temporary Variable
    k = np.arccos(-(g / (2 * i)))           # Helper Temporary Variable
    L = j * -1                              # Helper Temporary Variable
    M = np.cos(k / 3.0)                   # Helper Temporary Variable
    N = np.sqrt(3) * np.sin(k / 3.0)    # Helper Temporary Variable
    P = (b / (3.0 * a)) * -1                # Helper Temporary Variable

    x1 = 2 * j * np.cos(k / 3.0) - (b / (3.0 * a))
    x2 = L * (M + N) + P
    x3 = L * (M - N) + P

    return np.array([x1, x2, x3])           # Returning Real Roots as numpy array.
# Helper function to return float value of f.
def findF(a, b, c):
    return ((3.0 * c / a) - ((b ** 2.0) / (a ** 2.0))) / 3.0


# Helper function to return float value of g.
def findG(a, b, c, d):
    return (((2.0 * (b ** 3.0)) / (a ** 3.0)) - ((9.0 * b * c) / (a **2.0)) + (27.0 * d / a)) /27.0


# Helper function to return float value of h.
def findH(g, f):
    return ((g ** 2.0) / 4.0 + (f ** 3.0) / 27.0)

In [64]:
def calcFundamentalStats(y):
    ass = np.sum(y[1:-1]**2,axis=0)
    aep = y[0]**2 + y[-1]**2
    ac = np.sum(y[0:-1]*y[1:],axis=0)
    return aep,ass,ac

def calcBfromDataN(aep,ass,ac,N):
    coef = np.array([(N-1)*ass,
       (2.0-N)*ac,
       -aep-(N+1)*ass,
       N*ac])
    b = cubicsolver(coef)
    return b[2,:,:]

def calcAfromB(B,aep,ass,ac,N):
    Q=aep/(1-B**2)
    Q=Q+ass*(1+B**2)/(1-B**2)
    Q=Q-ac*2*B/(1-B**2)
    A = Q/N
    P2A = -N/A**2/2
    Btmp = B**2*(1+2*N)
    tmp = (1+Btmp)*aep + (2*Btmp + N + 1 -B**4*(N-1))*ass - 2*B*(1+B**2+2*N)*ac
    P2B = -tmp/((1-B**2)**2*(aep + (1+B**2)*ass - 2*B*ac))
    PAB = (N-1)*B/A/(1-B**2)
    dA = np.sqrt(-P2B/(P2A*P2B-PAB**2))
    dB = np.sqrt(-P2A/(P2A*P2B-PAB**2))
    return A,dA,dB

def calcCorr(windows,N,REGIONS):
    corrC = []
    corrdC = []
    corrB1 = []
    for m in windows:
        x1 = np.repeat(m[:, :, np.newaxis], REGIONS, axis=2)
        x2 = np.repeat(m[:, :, np.newaxis], REGIONS, axis=2).swapaxes(1,2)
        y1 = x1 + x2
        y2 = x1 - x2
        aep1,ass1,ac1 = calcFundamentalStats(y1)
        aep2,ass2,ac2 = calcFundamentalStats(y2)
        B1 = calcBfromDataN(aep1,ass1,ac1,N)
        B2 = calcBfromDataN(aep2,ass2,ac2,N)
        A1,dA1,dB1 = calcAfromB(B1,aep1,ass1,ac1,N)
        A2,dA2,dB2 = calcAfromB(B2,aep2,ass2,ac2,N)
        Adiff = A1-A2
        C = np.where(Adiff>0,Adiff/A2,Adiff/A1)
        dC = np.where(Adiff>0,np.sqrt(dA1**2/A1**2 + A1**2*dA2**2/A2**4),np.sqrt(dA2**2/A1**2 + A2**2*dA1**2/A1**4))
        corrC.append(C)
        corrdC.append(dC)
        corrB1.append(B1)
    return corrC,corrdC,corrB1

In [3]:
currentdir = os.getcwd()
datadir = os.path.join(currentdir,'OxytocinRSData_new')
print(datadir)

/Users/hstrey/Documents/programming/fMRI-analysis/OxytocinRSData_new


In [4]:
# get data files names
datafilenames = os.listdir(datadir)
datafilenames.sort()

In [5]:
time_series = np.load(os.path.join(datadir,datafilenames[1]),allow_pickle=True)
print(time_series.shape)

(748, 32)


In [46]:
# 748 = 2x2x11x17 so splitting it up into chuncks of 44 is a good idea
WINDOW_LENGTH =44
REGIONS = 32
num = int((time_series.shape[0] - 1 - \
               (time_series.shape[0] - 1) % WINDOW_LENGTH) /
              WINDOW_LENGTH)
indicies = np.linspace(WINDOW_LENGTH, num*WINDOW_LENGTH, num).astype(int)

windows = np.split(time_series, indicies, axis=0)

In [47]:
windows[0].shape

(44, 32)

In [65]:
C,dC,B1 = calcCorr(windows,WINDOW_LENGTH,REGIONS)

In [66]:
C[0]

array([[        nan, -0.10706633, -0.08582705, ...,  1.3621148 ,
         0.30110848, -0.3113514 ],
       [-0.10706633,         nan,  2.287269  , ..., -0.11149339,
        -0.3324413 , -0.1119395 ],
       [-0.08582705,  2.287269  ,         nan, ...,  0.4324632 ,
         0.50511056,  1.2791337 ],
       ...,
       [ 1.3621148 , -0.11149339,  0.4324632 , ...,         nan,
         0.64303637,  0.3700458 ],
       [ 0.30110848, -0.3324413 ,  0.50511056, ...,  0.64303637,
                nan,  2.549744  ],
       [-0.3113514 , -0.1119395 ,  1.2791337 , ...,  0.3700458 ,
         2.549744  ,         nan]], dtype=float32)

In [67]:
dC[0]

array([[       nan, 0.4811523 , 0.40897566, ..., 0.79884464, 0.38116524,
        0.4569949 ],
       [0.4811523 ,        nan, 1.3479211 , ..., 0.6421299 , 0.4880559 ,
        0.52075267],
       [0.40897566, 1.3479211 ,        nan, ..., 0.6877656 , 0.4092623 ,
        0.8124354 ],
       ...,
       [0.79884464, 0.6421299 , 0.6877656 , ...,        nan, 0.5636756 ,
        0.5995009 ],
       [0.38116524, 0.4880559 , 0.4092623 , ..., 0.5636756 ,        nan,
        0.8185537 ],
       [0.4569949 , 0.52075267, 0.8124354 , ..., 0.5995009 , 0.8185537 ,
               nan]], dtype=float32)

In [61]:
B1[0]

array([[0.47190538, 0.6040477 , 0.4290861 , ..., 0.75656635, 0.20665708,
        0.24578765],
       [0.6040477 , 0.69512045, 0.5928725 , ..., 0.75271153, 0.37707984,
        0.2998921 ],
       [0.4290861 , 0.5928725 , 0.47767687, ..., 0.68811786, 0.18454611,
        0.19061585],
       ...,
       [0.75656635, 0.75271153, 0.68811786, ..., 0.8216834 , 0.54425335,
        0.5862237 ],
       [0.20665708, 0.37707984, 0.18454611, ..., 0.54425335, 0.02299373,
        0.18487445],
       [0.24578765, 0.2998921 , 0.19061585, ..., 0.5862237 , 0.18487445,
        0.17468265]], dtype=float32)

In [63]:
dA1

array([[1.9185606 , 1.4092203 , 0.9736981 , ..., 0.8687135 , 1.6152146 ,
        1.2817225 ],
       [1.4092203 , 1.7349426 , 1.4585032 , ..., 0.85254246, 1.2805864 ,
        1.3689727 ],
       [0.9736981 , 1.4585032 , 1.1296115 , ..., 0.903061  , 1.079182  ,
        1.0356783 ],
       ...,
       [0.8687135 , 0.85254246, 0.903061  , ..., 3.2810636 , 0.5729598 ,
        0.7897095 ],
       [1.6152146 , 1.2805864 , 1.079182  , ..., 0.5729598 , 2.1465445 ,
        1.9803836 ],
       [1.2817225 , 1.3689727 , 1.0356783 , ..., 0.7897095 , 1.9803836 ,
        3.1825545 ]], dtype=float32)